### 1. Preprocessing

In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from preprocessing import combine_data_frompath


run this code below to see that you are using GPU for computing

In [ ]:

tf.config.list_physical_devices("GPU")

In [2]:
train_data = combine_data_frompath('aug_data\\train\\labels\\*.json','aug_data\\train\\images\\*.jpg')
val_data = combine_data_frompath('aug_data\\val\\labels\\*.json','aug_data\\val\\images\\*.jpg')

### 2. Create Model

In [3]:
from model import FaceTracker, gradient, compute_batch_loss
facetracker = FaceTracker()

In [ ]:
#facetracker.load_weights("./checkpoints/my_checkpoint")

### 3. Train Model

In [4]:

batches_per_epoch = len(train_data)
lr_decay = (1./0.75 -1)/batches_per_epoch
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)
num_epochs = 10

In [5]:
import datetime
# this is for the tensorboard we will plot the training loss in tensorboard
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/training/" + time 
writer_train = tf.summary.create_file_writer(logdir+"/train")
writer_val = tf.summary.create_file_writer(logdir+"/val")

for epoch in range(num_epochs):
    train_epoch_loss_avg = tf.keras.metrics.Mean()
    val_epoch_loss_avg = tf.keras.metrics.Mean()
    #epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    for x,y in train_data:
        loss_value, grads = gradient(facetracker,x,y)
        opt.apply_gradients(zip(grads, facetracker.trainable_variables))
        train_epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Use at the end of each epoch the loss for tensorboard
    with writer_train.as_default():
        tf.summary.scalar("train", train_epoch_loss_avg.result(), step=epoch+1)

    # same as above but instead compute the gradient we only compute the loss
    for x_val, y_val in val_data:
        val_loss = compute_batch_loss(facetracker,x_val,y_val,training=False)
        val_epoch_loss_avg.update_state(val_loss)
    with writer_val.as_default():
        tf.summary.scalar("train", val_epoch_loss_avg.result(), step=epoch+1)


    

### 4. Test Model

In [ ]:
#facetracker.save_weights('./checkpoints/my_checkpoint')

In [6]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step
